In [ ]:
pip install flask-sqlalchemy mysql-connector-python pymysql requests  

In [1]:
# Second Cell - Imports and Setup
from flask import Flask, jsonify, request
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
import threading
from threading import Thread
import time
import json

app = Flask(__name__)

# Configure MySQL connection - for root user with no password
app.config['SQLALCHEMY_DATABASE_URI'] = 'mysql+pymysql://root:@localhost/university_db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# Define Course model
class Course(db.Model):
    __tablename__ = 'course'
    
    subject_code = db.Column(db.Integer, primary_key=True, autoincrement=False)
    subject_name = db.Column(db.String(100), nullable=False)
    lecturer = db.Column(db.String(100), nullable=False)
    venue = db.Column(db.String(100), nullable=False)
    capacity = db.Column(db.Integer)

    def to_dict(self):
        return {
            'subject_code': self.subject_code,
            'subject_name': self.subject_name,
            'lecturer': self.lecturer,
            'venue': self.venue,
            'capacity': self.capacity
        }

# Create all tables
with app.app_context():
    db.create_all()


In [2]:
# Routes Definition
@app.route('/')
def home():
    return jsonify({"message": "Hello, API is working!"})

@app.route('/api/courses', methods=['POST'])
def create_course():
    try:
        if not request.is_json:
            return jsonify({"error": "Content-Type must be application/json"}), 400
        
        data = request.get_json()
        
        if not data or 'subject_code' not in data or 'subject_name' not in data or 'lecturer' not in data or 'venue' not in data or 'capacity' not in data:
            return jsonify({"error": "Missing required fields, Please provide all fields."}), 400
            
        new_course = Course(
            subject_code=data['subject_code'],
            subject_name=data['subject_name'],
            lecturer=data['lecturer'],
            venue=data['venue'],
            capacity=data['capacity'],
        )
        db.session.add(new_course)
        db.session.commit()
        return jsonify({"message": "Course created successfully", "course": new_course.to_dict()}), 201
    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 400

# Get all courses
@app.route('/api/courses', methods=['GET'])
def get_courses():
    try:
        courses = Course.query.all()
        return jsonify({"courses": [course.to_dict() for course in courses]})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Get course by Subject Id
@app.route('/api/course/<int:subject_code>', methods=['GET'])
def get_course(subject_code):
    try:
        course = Course.query.get(subject_code)
        if course is None:
            return jsonify({"error": "Course not found"}), 404
        return jsonify({"course": course.to_dict()})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Update course by Course Id
@app.route('/api/course/<int:subject_code>', methods=['PUT'])
def update_user(subject_code):
    try:
        if not request.is_json:
            return jsonify({"error": "Content-Type must be application/json"}), 400
            
        course = Course.query.get(subject_code)
        if course is None:
            return jsonify({"error": "Course not found"}), 404
            
        data = request.get_json()
        
        if 'subject_name' in data:
            course.subject_name = data['subject_name']
        if 'lecturer' in data:
            course.lecturer = data['lecturer']
        if 'venue' in data:
            course.venue = data['venue']
        if 'capacity' in data:
            course.capacity = data['capacity']
            
        db.session.commit()
        return jsonify({"message": "Course updated successfully", "course": course.to_dict()})
    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 400

# Delete a course
@app.route('/api/course/<int:subject_code>', methods=['DELETE'])
def delete_user(subject_code):
    try:
        course = Course.query.get(subject_code)
        if course is None:
            return jsonify({"error": "Course not found"}), 404
            
        db.session.delete(course)
        db.session.commit()
        return jsonify({"message": "Course deleted successfully"})
    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 400


In [ ]:
# Run the Flask app
def run_flask():
    app.run(port=5000)

thread = Thread(target=run_flask)
thread.daemon = True
thread.start()

# Give the server a moment to start
time.sleep(2)
print("Flask server is running on http://localhost:5000")


In [ ]:
# Test cell to verify server
import requests

try:
    response = requests.get('http://localhost:5000/')
    print(f"Server status: {response.status_code}")
    print(f"Response: {response.text}")
except requests.exceptions.ConnectionError:
    print("Error: Cannot connect to server. Make sure the Flask server is running.")

In [ ]:
# Test CRUD Operations
import requests
from pprint import pprint

base_url = 'http://localhost:5000/api'
headers = {'Content-Type': 'application/json'}

def test_api():
    # Test 1: Create a course
    print("\n1. Testing CREATE course:")
    create_data = {
        "subject_code":"112",
        "subject_name":"HND in Software Engineering",
        "lecturer":"John Smith",
        "venue":"Colombo Campus",
        "capacity":500
    }
    response = requests.post(
        f'{base_url}/courses',
        data=json.dumps(create_data),
        headers=headers
    )
    pprint(response.json())
    
    # Store the courses subject code
    subject_code = response.json().get('course', {}).get('subject_code')
    
    # Test 2: Get all courses
    print("\n2. Testing GET all courses:")
    response = requests.get(f'{base_url}/courses')
    pprint(response.json())
    
    # Test 3: Get specific course
    print(f"\n3. Testing GET course {subject_code}:")
    response = requests.get(f'{base_url}/course/{subject_code}')
    pprint(response.json())
    
    # Test 4: Update course
    print(f"\n4. Testing UPDATE course {subject_code}:")
    update_data = {
        "subject_code":"112",
        "subject_name":"HND in Computer Science & Software Engineering",
        "lecturer":"John Smith Updated",
        "venue":"Colombo Campus Updated",
        "capacity":1000
    }
    response = requests.put(
        f'{base_url}/course/{subject_code}',
        data=json.dumps(update_data),
        headers=headers
    )
    pprint(response.json())
    
    # Test 5: Delete course
    print(f"\n5. Testing DELETE course {subject_code}:")
    response = requests.delete(f'{base_url}/course/{subject_code}')
    pprint(response.json())
    
    # Test 6: Verify deletion
    print(f"\n6. Verifying deletion - trying to GET deleted course {subject_code}:")
    response = requests.delete(f'{base_url}/course/{subject_code}')
    pprint(response.json())
    
# Run the tests
test_api()